In [1]:
%load_ext autoreload
%autoreload 2

In [23]:
import torch
import torch_geometric as pyg
import torch_geometric.datasets as pygd
import warnings
from sklearn.exceptions import UndefinedMetricWarning

warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

In [52]:
from gpn.experiments.multiple_run_experiment import MultipleRunExperiment
from gpn.utils.config import (
    DataConfiguration,
    ModelConfiguration,
    RunConfiguration,
    TrainingConfiguration,
)


def create_experiment(model_name):
    run_cfg = RunConfiguration(
        job="train",
        eval_mode="default",
        experiment_directory="./.cache",
        save_model=True,
        gpu=0,
        experiment_name="ood_loc",
        num_inits=3,
        num_splits=10
    )
    data_cfg = DataConfiguration(
        dataset="CoraML",
        split_no=1,
        root="./data",
        ood_flag=True,
        train_samples_per_class=0.05,
        val_samples_per_class=0.15,
        test_samples_per_class=0.8,
        split="random",
        ood_setting="poisoning",
        ood_type="leave_out_classes",
        ood_num_left_out_classes=-1,
        ood_leave_out_last_classes=True,
    )

    model_cfg = ModelConfiguration(
        model_name=model_name,
        seed=42,
        init_no=1,
        dim_hidden=64,
        dropout_prob=0.5,
        K=10,
        add_self_loops=True,
        maf_layers=0,
        gaussian_layers=0,
        use_batched_flow=True,
        loss_reduction="sum",
        approximate_reg=True,
        flow_weight_decay=0.0,
        pre_train_mode="flow",
        alpha_evidence_scale="latent-new",
        alpha_teleport=0.1,
        entropy_reg=0.0001,
        dim_latent=16,
        radial_layers=10,
    )

    train_cfg = TrainingConfiguration(
        epochs=100000,
        stopping_mode="default",
        stopping_patience=50,
        stopping_restore_best=True,
        stopping_metric="val_CE",
        stopping_minimize=True,
        finetune_epochs=0,
        warmup_epochs=5,
        lr=0.01,
        weight_decay=0.001,
    )

    return MultipleRunExperiment(run_cfg, data_cfg, model_cfg, train_cfg)


gpn_e = create_experiment("GPN")
lop_e = create_experiment("GPN_LOP")

In [53]:
res = gpn_e.run()
res_lop = lop_e.run()

Early stopping after epoch 91 (patience 50).
Early stopping after epoch 107 (patience 50).
Early stopping after epoch 101 (patience 50).
Early stopping after epoch 96 (patience 50).
Early stopping after epoch 96 (patience 50).
Early stopping after epoch 101 (patience 50).


KeyboardInterrupt: 

In [51]:
import pandas as pd
from gpn.utils.utils import results_dict_to_df

df_lop = results_dict_to_df(res_lop)
df = results_dict_to_df(res)

pd.set_option("display.max_rows", None)
pd.DataFrame(
    dict(
        val_gpn=df["val"],
        test_gpn=df["test"],
        val_lop=df_lop["val"],
        test_lop=df_lop["test"],
    )
)

,val_gpn,test_gpn,val_lop,test_lop
accuracy,8.914530e-01,8.836773e-01,8.897436e-01,8.909944e-01
brier_score,2.734217e-01,2.607649e-01,4.586934e-01,4.495800e-01
ECE,7.267938e-02,5.667364e-02,2.420591e-01,2.539646e-01
confidence_aleatoric_apr,9.566202e-01,9.809136e-01,9.434349e-01,9.803639e-01
confidence_epistemic_apr,9.456113e-01,9.733324e-01,9.535282e-01,9.805961e-01
confidence_structure_apr,NaN,NaN,NaN,NaN
confidence_aleatoric_auroc,7.567282e-01,8.798027e-01,6.811286e-01,8.653210e-01
confidence_epistemic_auroc,7.068891e-01,8.419667e-01,7.455218e-01,8.756331e-01
confidence_structure_auroc,NaN,NaN,NaN,NaN
CE,4.044366e-01,3.410517e-01,5.925173e-01,5.761619e-01
